<a href="https://colab.research.google.com/github/LeNovice/Assignments_WILP_Sem1/blob/master/FINAL_v0_Assignment1_RBFS_DLS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial and Computational Intelligence Assignment 1

## Problem solving by Uninformed & Informed Search

List only the BITS (Name) of active contributors in this assignment:
1. WADIKAR PINAK SUHAS [2020sc04158]
2. NITESH GARG [2020SC04748]
3. MANOJ KUMAR SHARMA [2020SC04723]
4. SABYASACHI SAMADDAR [2020SC04167]


Things to follow
1.	Use appropriate data structures to represent the graph and the path using python libraries
2.	Provide proper documentation
3.	Find the path and print it

Coding begins here

### 1.	Define the environment in the following block

# PEAS Description:

The selected Network agent is a Goal-based PSA. Following are the PEAS factors which are considered while preparing this agent.

###Performance Measure:
* Minimizing the Total cost of the generated subgraph
* Nodes are connected in the subgraph (no isolated node)
* Acheiving a better signal to noise ratio along with minimizing the delay between sender and receiver

###Environment:
* Uplink/Downlink from connected nodes
* Valid ping between all the nodes
#####**Assumptions:**

*   Fully Observable: The heuristics between nodes are calculated initially and are known to the Agent
*   Deterministic: No stochastic/Random variables involved
*   Static: Heuristics do not change during any action performed by the agent
*   Discrete: State of environment is governed by heuristic/number of connections, both discrete





###Actuators:
* Ping distortion (delay or noise) to emulate the effect of heuristic between two nodes. 
* Counter to measure the number of active connections in any subgraph
* Display input as well as goal node
Hubs, routers, Master controllers to maintain ping between nodes

###Sensors:
* Ping distortion measuring devices, based on the distortion caused by the heuristic.
* Signal to Noise Ratio detectors for noise distortions
* Skew detectors for Delay distortion.
* Keyboard entries


##Informed Search: Recursive Best First Search
* Best First search requires a goal node which we take in as input from the user to calculate the heuristic value for each node, as per problem definition

Q] Design the agent as PSA Agent(Problem Solving Agent) 
Clear Initial data structures to define the graph and variable declarations is expected 
IMPORTANT: Write distinct code block as below

In [442]:
import sys
from heapq import heappush, heappop
import copy

MAX_DEPTH = 0
heuristic_dict = {}
ISOLATED_PENALTY = 20
CYCLIC_PENALTY = ISOLATED_PENALTY
MAX_HEUR = 999

In [443]:
#Definition of State: Every node in the network corresponds to a State
class State:
    def __init__(self, Name, Graph, Prev_state=None, Action=None, Heap=[], init_node = 0, DFS_Flag=0):
        self.name = Name
        self.graph = copy.deepcopy(Graph)
        self.prev_state = Prev_state
        self.heap = Heap                    #State heap will be stored to avoid future re - calculations.
        self.AllNodesVisited = 0
        if (init_node == 1):
            self.h = MAX_HEUR
            self.g = 0
            self.depth=0                        #initial state
#            self.action = [].append(self.name)
        else:
            self.h = self.CalcHeur()
            self.g = self.CalcPathCost()     #We can find this from graph
            Prev_state = self.prev_state
            self.depth = Prev_state.depth + 1
#            self.action = self.prev_state.action.append(Action)
        self.f = self.h + self.g
    #Override comparison functions to compare the states based on their f_value
    def __eq__(self, other):
        if (self.f == other.f):
            return 1
        else:
            return 0
    def __ge__(self,other):
        if (self.f >= other.f):
            return 1
        else:
            return 0
    def __le__(self,other):
        if (self.f <= other.f):
            return 1
        else:
            return 0
    def __lt__(self,other):
        if (self.f < other.f):
            return 1
        else:
            return 0
    def __gt__(self,other):
        if (self.f > other.f):
            return 1
        else:
            return 0
    def __ne__(self,other):
        if (self.f != other.f):
            return 1
        else:
            return 0
    #Functions to calculate h(n) for this state
    def CalcHeur(self):
        """
        Heuristics of a state is calculated based on how many nodes are isolated. we penalize 26 units on number of non-connected nodes
        This is just large of the maximum edge cost in the graph
        In order to keep the number of edges minimum, we alo need to add a heuristic to penalize the cost of existing connections however we will look at this penalty as pathcost (stepcost/ cost required to do the action) i.e. g(n) and not add it in h(n)
        Another part where we penalize the code is the number of cycles in the graph
        """
        current_graph = self.graph
        #Isolated nodes penalty
        isolated_penalty = 0
        for key in current_graph:
            conn = current_graph[key]
            if(len(conn) == 0):
                isolated_penalty += ISOLATED_PENALTY
        #Cycle penalty
        node_name = self.name
        if(isCyclic('Z', node_name, [], current_graph)):
            print("Graph is Cyclic!")
            return isolated_penalty + CYCLIC_PENALTY
        else:
            #print("Graph is NOT Cyclic!")
            return isolated_penalty
    #Function to calculate g(n) for this state
    def CalcPathCost(self):
        """
        This is the path cost. Everytime the agent adds another edge to the problem, the additional edge will have some penalty. We consider this as the path cost (g(n))to go towards goal node (goal node where all nodes are connected and h(n) = 0)
        Usually this is calculated while traversal but due to the problem defintion, we can calculate this statically
        Since the graph is bidirectional, the edges will be calculated twice. We will divide by 2 for this
        """
        current_graph = self.graph
        connected_cost = 0
        for key in current_graph:
            conn = current_graph[key]
            if(len(conn) != 0):
                for node, edgecost in conn:
                    connected_cost += edgecost
        return connected_cost/2
def isCyclic(parent_node_name, node_name, VisitedList, thegraph):
    succ_list = thegraph[node_name]
#    print(thegraph)
    #if i is parent_node, that is not considered to be a cycle since this is a bidirectional graph so go ahead with the flow
    for i,val in succ_list:
        if(i != parent_node_name):
#            print(i)
#            print("Visited List = " +str(VisitedList))
            if(i in VisitedList):
                return True
            else:
                #add i in visited list and expand i
                VisitedList.append(i)
                new_visited_list = copy.deepcopy(VisitedList)
                ret = isCyclic(node_name, i, new_visited_list, thegraph)
                if ret==True:
                    #If cycle has been detected, return the True value
                    return ret
                #Otherwise continue with the next node  
         

class State_DFS:
    def __init__(self, name, Graph, Prev_node=None, Action=None, Cost=999):
        
        self.name = name
        self.graph = Graph  #Also the status
        self.prev_node = Prev_node
        self.action = Action
        self.cost = Cost
        self.AllNodesVisited = 0
        if (self.prev_node == None):
            self.depth=0    #initial state
        else:
            Prev_state = self.prev_node
            self.depth = Prev_state.depth + 1


In [444]:
#Code Block : Set the matrix for transition & cost (as relevant for the given problem)
graph = {
    'A' : [('B',15),('D',17),('C',10),('G',5)],
    'B' : [('A',15),('D',12)],
    'C' : [('A',10),('G',7)],
    'D' : [('A',17),('B',12),('E',2),('F',10),('H',4)],
    'E' : [('D',2)],
    'F' : [('D',10),('H',11)],
    'G' : [('A',5),('C',7),('H',25)],
    'H' : [('D',4),('G',25),('F',11)]
}
graph_init ={
    'A' : [],
    'B' : [],
    'C' : [],
    'D' : [],
    'E' : [],
    'F' : [],
    'G' : [],
    'H' : []
}
#____________________________________________________DFS variables________________________________________________________________
graph_DFS = {
    'A' : ['B','D','C','G'],
    'B' : ['A','D'],
    'C' : ['A','G'],
    'D' : ['A','B','E','F','H'],
    'E' : ['D'],
    'F' : ['D','H'],
    'G' : ['A','C','H'],
    'H' : ['D','G','F']
}

def graph_AddEdge(state, edge):
    """
    edge is expected to be in the form of (dstnodeB, edgecost)
    """
    curr_graph = state.graph
    curr_nodename = state.name
    nodeB = edge[0]
    edgecost = edge[-1]
    for key, conn in curr_graph:
        if (key == curr_nodename):
            for node_name, edgecost in conn:
                if (node_name == nodeB):
                    sys.exit("GRAPH_ADDEDGE: Connection from " + curr_nodename + " to " + nodeB + " already exists!")
    curr_graph[curr_nodename].append(edge)
    return curr_graph

def Go(current_node_state, new_node_name, absHeur, algo):
    """
    We call this function when our algorithm has decided on which node to traverse to based on whether it is Best First or any other algo..
    This function associates State to the new node and returns state
    This is a function specific to the state, can be integrated under the same class
    """
    curr_node_name = current_node_state.name
    curr_graph = copy.deepcopy(current_node_state.graph)
    if (new_node_name in curr_graph[curr_node_name]):
        #If new_node_name is in currently visited list name then it seems like we have hit a loop for example: A->G->C->A Curr= C, new_node = A
        #We return None and add checks in the calling function. For recursive functions, this means we have to go to previous state, i.e. G
        print("GO: New node in current visited list")
        return None
    else:
        #curr_visited_list.append(new_node_name)
#        print("Before \n" + str(curr_graph))
        curr_graph[curr_node_name].append(new_node_name)
        curr_graph[new_node_name].append(curr_node_name)
        new_graph = copy.deepcopy(curr_graph)
#        print("after \n" + str(curr_graph))
        return State_DFS(new_node_name, new_graph, current_node_state, "Go(" + new_node_name + ")" , 1)


In [445]:
#Code Block : Write function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented

def succ_fn(node, algo):
    """
    #input: Current state of network Agent
    #output: A heap of Action, Heuristic. For example, ["Go(B)", (heuristic)]
    #Transition Model will return sorted list of possible nodes that we can visit from current node.
    The output is in the form of a min-heap. Each element is (possible_successor_node_name, Heuristic)
    """
    heap = []
    curr_node_name = node.name
    
    #Explore graph to see what the successors of this node are and also get corresponding heuristic
    if (algo == "BFS"):
        curr_graph = node.graph
        succ_list = graph[curr_node_name]
        #Above will give you all possible connections from current node. This is from main graph
        for node_edgecost in succ_list:
            if (node_edgecost not in curr_graph[curr_node_name]):
                new_node_name = node_edgecost[0]
                edgecost = node_edgecost[1]
                temp_curr_graph = copy.deepcopy(curr_graph)
#                if(node.g == 0):
                    #This is initial node. Expand all successors unconditionally
                    #Add node from curr node to new node
                temp_curr_graph[curr_node_name].append(node_edgecost)
                    #Similarly add node from new node to curr node
                temp_curr_graph[new_node_name].append((curr_node_name,edgecost))
#                print(curr_graph)
#                print("node_edgecost = " + str(node_edgecost))
#                print(temp_curr_graph)
                new_graph = temp_curr_graph
                heappush(heap, State( new_node_name, new_graph, node, "(" + curr_node_name + "->" + new_node_name + ")"))
#                elif(new_node_name != node.prev_state.name):
#                    temp_curr_graph[curr_node_name].append(node_edgecost)
#                    temp_curr_graph[new_node_name].append((curr_node_name,edgecost))
#                    new_graph = temp_curr_graph
#                    heappush(heap, State( new_node_name, new_graph, node, "(" + curr_node_name + "->" + new_node_name + ")"))
        return heap
    elif (algo == "DFS"):
        succ_list = graph_DFS[curr_node_name]
        return sorted(succ_list)
   

In [446]:
#Code block : Write fucntion to handle goal test (Must handle dynamic inputs). Ideally this would be called while search algorithms are implemented
def isGoalTest(node_state):
    return (node_state.h ==0)

"""
def prep_goal_state():
    #Goal state is independent of inputs, Our goal state is to include all the nodes in the communication network
    goal_list = []
    for key in graph:
        goal_list.append(key)
    return sorted(goal_list)

def isGoalTest_DFS(node_state):
    goal_list = prep_goal_state()                   #fixme: No need to call this for every comparison
    current_visited_set = set(node_state.visited_list)

    if len(current_visited_set) != len(goal_list):
        return False
    return (sorted(current_visited_set) == sorted(goal_list))
"""
def isGoalTest_DFS(node_state):
    current_graph = node_state.graph
    #print(current_graph)
    for key in current_graph:
        conn = current_graph[key]
        if(len(conn) == 0):
            return False
    return True

### 2.	Definition of Algorithm 1 
INFORMED SEARCH - RECURSIVE BEST FIRST SEARCH)

In [447]:
#Code Block : Function for algorithm 1 implementation
expanded_list = []
RECURSION_LIM = 100
REC_COUNT = 0
def RBFS(node_state, f_limit):
    """
    Assumes that you have already calculated te successor functions of node_state before calling this function.
    For every new node encounteed, you need to do following steps:
    new_node_state = Go(new_node)
    new_node_state.heap = succ_fn(new_node_state)
    BestFirst(new_node_state)
    """
    print("expanding " + node_state.name +" with f_limit = " + str(f_limit))

    global MAX_DEPTH
    global MAX_HEUR
    if(isGoalTest(node_state)):
        print("Goal!!")
        node_state.AllNodesVisited=1
        return node_state, node_state.f
    
    if (len(node_state.heap) > 0):
        #This means that this state was previously visited
        succ_heap = node_state.heap
    else:
        succ_heap = succ_fn(node_state, "BFS")

    
    single_flag=0
    display_heap = copy.deepcopy(succ_heap)
    if len(display_heap)==1:
        sing_node = heappop(display_heap)
        if (sing_node.name == node_state.prev_state.name):
            print(sing_node.name + " lead to its parent, i.e. " + node_state.prev_state.name + "This qualifies as a singular node")
            #sing_node is definitely a singular node with no successors (only one successor which is its prev node itself)
            single_flag=1
        heappush(display_heap, sing_node)
    #DEBUG Start
    while(len(display_heap)>0):
        i = heappop(display_heap)
        print(i.name + ":" + str(i.f), end = '\t')
        
    #DEBUG End

    global REC_COUNT
    REC_COUNT = REC_COUNT + 1
    if (REC_COUNT>=RECURSION_LIM):
        sys.exit("Too many recursions")

    while (len(succ_heap) > 0):
        best_state = heappop(succ_heap)
        best_f = best_state.f

        if (best_f > f_limit):             #>= has been added. equal condition has been added to avoid a going back to parent node after a singluar node (like E) has been expanded (because when E is expanded, )
            node_state.heap = succ_heap
            return best_state, best_f

        if (len(succ_heap) == 0):
            alternative_f = f_limit
            #Create a bogus alternative state to get rid of "local variable 'alternative_state' referenced before assignment"
            alternative_state = State( "XY", graph_init, Prev_state=None, Action=None, Heap=[], init_node = 1)
            alternative_state.h = MAX_HEUR
            alternative_state.g = MAX_HEUR
            alternative_state.f = MAX_HEUR
            print("\nbest = " + best_state.name + " : " + str(best_f) + "\tAlternative = NONE" + " : " + str(alternative_f))
        else:
            alternative_state = heappop(succ_heap)
            alternative_f = alternative_state.f
            print("\nbest = " + best_state.name + " : " + str(best_f) + "\tAlternative = " + alternative_state.name + " : " + str(alternative_f))

        result, new_best_f = RBFS(best_state, min(f_limit,alternative_f))
        MAX_DEPTH = max(MAX_DEPTH,result.depth)
        print("\nReturned from " + best_state.name + " to " + node_state.name + " With new f_value value : " + str(new_best_f))

        if (result.AllNodesVisited==0):             #Equivalent to: if result==failure
            if(new_best_f != MAX_HEUR):
                #We push Alternative state before best state so that in case both have same value, the alternative case will be picked first (to avoid looping)
                #Also push the alternative value into heap which was popped earlier. This will be your new best value which will be popped at next iteration of while
                if(single_flag == 0):
                    #Only add alternative state if it existed in first case.
                    #print("Alternative state " + alternative_state.name + " with f_value = " + str(alternative_f) + " pushed back into heap")
                    heappush(succ_heap, alternative_state)
                else:
                    print("Seems like all successors of " + node_state.name + " have been visited, breaking....")
                    #if current node has only one successor, we have visited it via best. so now we must return to parent and not come back to this
                    node_state.heap = succ_heap
                    break
                #update the best state with the new best f_value and push the best node with the updated f_value back into successor list
                best_state.f = new_best_f
                heappush(succ_heap, best_state)
            elif(new_best_f == MAX_HEUR):
                node_state.graph = copy.deepcopy(result.graph)
                #node_state.f = result.f                     #CHECK!!!
                #We will also have to recalculate the successor list because now, for the successors, the singular node will be connected (for h(n)) calculation
                succ_heap = succ_fn(node_state, "BFS")
        elif(result.AllNodesVisited==1):        #Goal node has been achieved in recursions of this path
            node_state.heap = succ_heap
            return result, new_best_f
    #If succ_heap is empty
    node_state.heap = succ_heap
    return node_state, MAX_HEUR             #node_state.f

### 3.	Definition of Algorithm 2 

UNINFORMED SEARCH: DEPTH LIMITEd SEARCH

In [448]:
#Code Block : Function for algorithm 2 implementation
def DepthLimited(node_state, limit):
    """
    Assumes that you have already calculated te successor functions of node_state before calling this function.
    For every new node encounteed, you need to do following steps:
    new_node_state = Go(new_node)
    new_node_state.heap = succ_fn(new_node_state)
    DepthLimited(new_node_state)
    """
    #Check if current node state is Goal node
    if(isGoalTest_DFS(node_state)):
        for key in node_state.graph:
            conn = node_state.graph[key]
            print(key + ":\t" + str(conn))
        node_state.AllNodesVisited=1
        return node_state
    else:
        #Get all the successor functions for the current state:
        curr_name = node_state.name
        
        #print("Expanding :" + str(curr_name))
        succ_heap = succ_fn(node_state, "DFS")
        #print("succ_heap\t" + str(succ_heap))
        
        while ((len(succ_heap) > 0) and (limit > 0)):
            curr_graph = copy.deepcopy(node_state.graph)
            next_node_name = heappop(succ_heap)
            if (next_node_name not in curr_graph[curr_name]):
                #New Node encountered. Create a state for it
                next_node_state = Go(node_state, next_node_name, 1, "DFS")
                if(next_node_state is None):
                    sys.exit("DFS ALGO: " + next_node_name + " found in visited list")
                else:
                    #Apply BFS recursively on next node
                    ret = DepthLimited(next_node_state, limit-1)
                    #print("returned from " + next_node_state.name + " to " + curr_name)
                    if(isinstance(ret, State_DFS)):
                        if(ret.AllNodesVisited == 1):
                            return ret
                    #else:

                        else:
                            updated_curr_graph = copy.deepcopy(ret.graph)
                            #curr_graph.append(node_state.name)
                            node_state.graph =  updated_curr_graph = copy.deepcopy(ret.graph)

        return node_state

### DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question. 

In [449]:
#Code Block : Function & call to get inputs (start/end state)
init_node_name = input("Please enter the start node (A,B,C,D,E,F,G,H):").upper();
#goal_node_name = 'A'#input("Please enter the goal node upto which the path needs to be printed (A,B,C,D,E,F,G,H):").upper();


Please enter the start node (A,B,C,D,E,F,G,H):C


### 4.	Calling the search algorithms
(For bidirectional search in below sections first part can be used as per Hint provided. Under second section other combinations as per Hint or your choice of 2 algorithms can be called .As an analyst suggest suitable approximation in the comparitive analysis section)

In [450]:
#Code Block : Set Initial State (Must handle dynamic inputs)
dummy_Prev_state = State('Y', graph_init, Prev_state=None, Action=[], Heap=[], init_node=1)
dummy_Prev_state.f =MAX_HEUR
dummy_Prev_state.g =MAX_HEUR
dummy_Prev_state.h =MAX_HEUR

init_state = State(init_node_name, graph_init, dummy_Prev_state, Action=[], Heap=[], init_node=1)

In [451]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))
MAX_DEPTH = 0
ret, f_val = RBFS(init_state, MAX_HEUR)
if (ret is None):
    print("Could not find a sub graph which connects all nodes")
elif (isinstance(ret, State)):
    if(ret.AllNodesVisited == 1):
        print("Goal Achieved!\nOptimal Sub Graph:\n")
        Optimal_Graph = ret.graph
        for key in Optimal_Graph:
            conn = Optimal_Graph[key]
            print(key + ":\t" + str(conn))
        print("Max Depth = " +str(MAX_DEPTH))
    else:
        print("Could not find a sub graph which connects all nodes")

expanding C with f_limit = 999
G:127.0	A:130.0	
best = G : 127.0	Alternative = A : 130.0
expanding G with f_limit = 130.0
A:112.0	H:132.0	
best = A : 112.0	Alternative = H : 132.0
expanding A with f_limit = 130.0
Graph is Cyclic!
B:107.0	D:109.0	C:142.0	
best = B : 107.0	Alternative = D : 109.0
expanding B with f_limit = 109.0
D:99.0	
best = D : 99.0	Alternative = NONE : 109.0
expanding D with f_limit = 109.0
Graph is Cyclic!
E:81.0	H:83.0	F:89.0	A:136.0	
best = E : 81.0	Alternative = H : 83.0
expanding E with f_limit = 83.0

Returned from E to D With new f_value value : 999
Graph is Cyclic!

best = H : 65.0	Alternative = F : 71.0
expanding H with f_limit = 71.0
Graph is Cyclic!
F:56.0	G:110.0	
best = F : 56.0	Alternative = G : 110.0
expanding F with f_limit = 71.0
Goal!!

Returned from F to H With new f_value value : 56.0

Returned from H to D With new f_value value : 56.0

Returned from D to B With new f_value value : 56.0

Returned from B to A With new f_value value : 56.0

Returned

In [452]:
#Invoke algorithm 2 (Should Print the solution, path, cost etc., (As mentioned in the problem))
#init_state = State_DFS(init_node_name, graph_init, Prev_state=None,  Action=[], Heap=[], init_node = 1, DFS_Flag=1)
init_state = State_DFS(init_node_name, graph_init, Prev_node=None, Action=None, Cost=999)
LIMIT = 1
MAX_LIMIT = 5
ret = DepthLimited(init_state, LIMIT)
#if (ret is None):
#    print("Could not find a sub graph which connects all nodes")
#elif (isinstance(ret, State)):
#    if(ret.AllNodesVisited == 1):
#        print("Goal Achieved!\nOptimal Sub Graph:\n")
#        print(ret.visited_list)
#    else:
#        print("Cutoff achieved\n Following is the incomplete graph")
#        print(ret.visited_list)
#print("graph_init = " + str(graph_init))
while((ret.AllNodesVisited != 1) and (LIMIT<MAX_LIMIT)):
    init_state = State_DFS(init_node_name, graph_init, Prev_node=None, Action=None, Cost=999)
    print("Attempt with LIMIT = " + str(LIMIT))
    ret = DepthLimited(init_state, LIMIT)
    if (ret is None):
        sys.exit("Something Went Wrong! Could not find a sub graph which connects all nodes")
    elif (isinstance(ret, State_DFS)):
        if(ret.AllNodesVisited == 1):
            print("Goal Achieved!\nOptimal Sub Graph shown in the form of adjacency list:")
            Optimal_subgraph = ret.graph
            for key in Optimal_subgraph:
                conn = Optimal_subgraph[key]
                print(key + ":\t" + str(conn))
            print("LIMIT_Value = " + str(LIMIT))
        else:
            print("Cutoff achieved\n Following is the incomplete graph")
            Final_subgraph = ret.graph
            for key in Final_subgraph:
                conn = Final_subgraph[key]
                print(key + ":\t" + str(conn))
            print("Retrying with larger LIMIT value ....\n")
            LIMIT = LIMIT + 1

Attempt with LIMIT = 1
Cutoff achieved
 Following is the incomplete graph
A:	['C']
B:	[]
C:	['A', 'G']
D:	[]
E:	[]
F:	[]
G:	['C']
H:	[]
Retrying with larger LIMIT value ....

Attempt with LIMIT = 2
Cutoff achieved
 Following is the incomplete graph
A:	['C', 'B', 'D', 'G']
B:	['A']
C:	['A', 'G']
D:	['A']
E:	[]
F:	[]
G:	['A', 'C', 'H']
H:	['G']
Retrying with larger LIMIT value ....

Attempt with LIMIT = 3
A:	['C', 'B', 'D', 'G']
B:	['A', 'D']
C:	['A']
D:	['B', 'A', 'E', 'F', 'H']
E:	['D']
F:	['D']
G:	['A']
H:	['D']
Goal Achieved!
Optimal Sub Graph shown in the form of adjacency list:
A:	['C', 'B', 'D', 'G']
B:	['A', 'D']
C:	['A']
D:	['B', 'A', 'E', 'F', 'H']
E:	['D']
F:	['D']
G:	['A']
H:	['D']
LIMIT_Value = 3


###How to read above optimal subgraph:

For input node C, we have following Optimal Sub graph at Depth Limit = 3

Optimal Sub Graph:

{

A:	['C', 'B', 'D', 'G']

B:	['A', 'D']

C:	['A']

D:	['B', 'A', 'E', 'F', 'H']

E:	['D']

F:	['D']

G:	['A']

H:	['D']

}

LIMIT_Value = 3

Above List shows the connection between every node to every other node. The connections constitute an MST. At depth limit =3 i.e. at 3 recursion depth we see that all the nodes are connected in a single sub graph which is our goal state.

### 5.	Comparitive Analysis

In [453]:
#Code Block : Print the Time & Space complexity of algorithm 1 
curr_max = 0
for key in graph:
    curr_max = max(curr_max, len(list(graph[key])))
branching_factor = curr_max
print("branching_factor = " + str(branching_factor))
print("MAX_DEPTH of the fully connected sub graph: " + str(MAX_DEPTH))
print("Time Complexity of Recursive Best First Search = Big-Oh(b^l): " + str(branching_factor ** MAX_DEPTH))
print("Space Complexity of Recursive Best First Search = Big-Oh(b*l): " + str(branching_factor*MAX_DEPTH))

branching_factor = 5
MAX_DEPTH of the fully connected sub graph: 6
Time Complexity of Recursive Best First Search = Big-Oh(b^l): 15625
Space Complexity of Recursive Best First Search = Big-Oh(b*l): 30


In [454]:
#Code Block : Print the Time & Space complexity of algorithm 2
curr_max = 0
for key in graph_DFS:
    curr_max = max(curr_max, len(list(graph_DFS[key])))
branching_factor = curr_max
print("branching_factor = " + str(branching_factor))
print("LIMIT at which fully connected sub graph was achieved: " + str(LIMIT))
print("Time Complexity of Depth Limited Search (at limit LIMIT) = Big-O(b^l)", branching_factor**LIMIT)
print("Space Complexity of Depth Limited Search (at limit LIMIT) = Big-O(b*l)", branching_factor*LIMIT)

branching_factor = 5
LIMIT at which fully connected sub graph was achieved: 3
Time Complexity of Depth Limited Search (at limit LIMIT) = Big-O(b^l) 125
Space Complexity of Depth Limited Search (at limit LIMIT) = Big-O(b*l) 15


### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section

Comparison : Apart from time and space complexity, in terms of optimal and completeness, recursive Best First is better that Depth Limited Search.

###|Algorithm| Optimal | Completeness|

|BFS| Yes | Yes |

|DFS| No | No |

##**For Start Node: C and Stop node: A**
###**Optimal connected Sub Graph of Recursive Best First Search:**

['C', 'A', 'G', 'H', 'D', 'E', 'D', 'F', 'D', 'B']

###**Optimal connected Sub Graph of Depth Limited Search:**

['C', 'A', 'B', 'D', 'E', 'D', 'F', 'D', 'H', 'D', 'B', 'A', 'G']